In [1]:
import numpy as np
import tensorflow as tf
import tensorflow
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten, Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Model

In [ ]:
################################################
### TRIAL 1
################################################

# Config
#tf.compat.v1.disable_eager_execution()

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# We will create our own dataset
data = ['I would like to close my account. My account number is 000345-9876',
       'Hello I would like to close my account',
       'My account needs to be closed',
       'I need to close my account',
       'Can you close my account. My SSN is 345-456-88976',
       'My name is Anant Gupta and I would like to close my account',
       'I need to open a new account. My SSN is 607-089-4563',
       'I need to open a new account. Do you need my SSN'
       ]
labels = [1,0,0,0,1,0,1,0]

# We will now be converting SSN numbers to SSSSS, ACCOUNT numbers to XXXXX
import re
data = [re.sub(r"\d\d\d-\d\d\d-\d\d\d\d", "SSSSS", x) for x in data]
data = [re.sub(r"\d\d\d\d\d\d-\d\d\d\d", "XXXXX", x) for x in data]
data = [str(x).lower().replace('.',' ') for x in data]
y_train = np.array(labels)

# Unfortunately this cannot be used
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_data = np.unique([str(z).lower() for x in data for y in x.split('.') for z in y.split(' ') if z!= ''])
max_len = len(vocab_data)
vectorize_layer = TextVectorization(
    max_tokens=max_len,
    output_mode='binary')
# Create the layer so that it can be used
vectorize_layer.adapt(np.asarray(data))

# Simplest Dense Layer
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Dense(1, activation="sigmoid"))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)

# Fit the model
model.fit(np.array(data).reshape(-1,1), y_train, epochs=1, verbose=True)

# Save the model
tf.keras.models.save_model(model, "pii_model_20210426", overwrite=True, save_format='tf')
model.save("pii_model_20210426_format1")

In [17]:
################################################
### TRIAL 2
################################################

# Config
#tf.compat.v1.disable_eager_execution()

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# We will create our own dataset
data = ['I would like to close my account. My account number is 000345-9876',
       'Hello I would like to close my account',
       'My account needs to be closed',
       'I need to close my account',
       'Can you close my account. My SSN is 345-456-88976',
       'My name is Anant Gupta and I would like to close my account',
       'I need to open a new account. My SSN is 607-089-4563',
       'I need to open a new account. Do you need my SSN'
       ]
labels = [1,0,0,0,1,0,1,0]

# We will now be converting SSN numbers to SSSSS, ACCOUNT numbers to XXXXX
import re
data = [re.sub(r"\d\d\d-\d\d\d-\d\d\d\d", "SSSSS", x) for x in data]
data = [re.sub(r"\d\d\d\d\d\d-\d\d\d\d", "XXXXX", x) for x in data]
data = [str(x).lower().replace('.',' ') for x in data]
vocab_data = np.unique([str(z).lower() for x in data for y in x.split('.') for z in y.split(' ') if z!= ''])
print(data)
y_train = np.array(labels)


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)
X = X.toarray()
X[X>0]=1
print(X)
print(vectorizer.vocabulary_)

# Simplest Dense Layer
model = tf.keras.models.Sequential()
model.add(Dense(1, activation="sigmoid"))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)

# Fit the model
model.fit(X.astype(np.float32), y_train, epochs=3000, verbose=False)
predictions = model.predict(X.astype(np.float32)).reshape(-1)
predictions = [1 if x>0.5 else 0 for x in predictions]
print(confusion_matrix(y_train, predictions))

# Save the model
tf.keras.models.save_model(model, "pii_model_20210426.h5", overwrite=True, save_format='keras')

!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model pii_model_20210426.h5 pii_model_20210426_tfjs

max_length = np.max(list(vectorizer.vocabulary_.values()))
vocab_dict = dict((vectorizer.vocabulary_[i],i) for i in vectorizer.vocabulary_.keys())
vocab=[]
for i in range(max_length+1):
    vocab.append(vocab_dict[i])

['i would like to close my account  my account number is xxxxx', 'hello i would like to close my account', 'my account needs to be closed', 'i need to close my account', 'can you close my account  my ssn is sssss6', 'my name is anant gupta and i would like to close my account', 'i need to open a new account  my ssn is sssss', 'i need to open a new account  do you need my ssn']
[[1 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0]
 [1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1]
 [1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1]]
{'would': 23, 'like': 11, 'to': 22, 'close': 5, 'my': 12, 'account': 0, 'number': 17, 'is': 10, 'xxxxx': 24, 'hello': 9, 'needs': 15, 'be': 3, 'closed': 6, 'need': 14, 'can': 4, '